In [7]:
# ✅ Step 1.1: Import Libraries
import pandas as pd
import numpy as np

# ✅ Step 1.2: Load CSV files
try:
    df_clusters = pd.read_csv(r"C:\Users\Administrator\Desktop\Python\ML\ProjectPlan\KNN-SubClusters\knn_subclusters.csv")
    df_vehicles = pd.read_csv(r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map_with_capacity.csv")
    print("✅ Files loaded successfully.")
except FileNotFoundError as e:
    print("❌ File not found:", e)
except Exception as e:
    print("❌ Unexpected error:", e)

# ✅ Step 1.3: Quick preview
try:
    print("\n🧾 Cluster Data Preview:")
    print(df_clusters.head(3))

    print("\n🚚 Vehicle Data Preview:")
    print(df_vehicles.head(3))
except Exception as e:
    print("❌ Error while printing previews:", e)


✅ Files loaded successfully.

🧾 Cluster Data Preview:
                    Assignment_ID        Product_ID  Urgent_Flag  \
0  ZONE-4::RACK-1::795::GJ08X2709  DEW-1000-B31-795            0   
1  ZONE-2::RACK-1::384::GJ30X2371  IMM-1001-B59-384            1   
2  ZONE-4::RACK-1::434::GJ24X8239  SUP-1002-B44-434            0   

  Dispatch_Window Delivery_Window  Unit_Weight (kg)  Unit_Volume (L)  \
0      2025-07-03      2025-07-05               0.6             0.25   
1      2025-07-04      2025-07-08               1.0             0.60   
2      2025-07-04      2025-07-07               2.5             1.20   

   Fragile_Flag  Temp_Sensitive_Flag  Quantity_Assigned  Total_Weight  \
0             0                    0                 37          22.2   
1             0                    1                 37          37.0   
2             0                    0                 20          50.0   

   Total_Volume  Priority_Label  Cluster_Label  Vehicle_ID Main_Cluster_Label  \
0         

In [8]:
# ✅ Step 2.1: Copy DataFrames
try:
    orders_df = df_clusters.copy()
    vehicles_df = df_vehicles.copy()
    print("✅ DataFrames copied.")
except Exception as e:
    print("❌ Error copying DataFrames:", e)

# ✅ Step 2.2: Add ID Columns (optional but useful for RL context)
try:
    orders_df["Order_ID"] = orders_df.index
    vehicles_df["Vehicle_Index"] = vehicles_df.index
    print("✅ ID columns added.")
except Exception as e:
    print("❌ Error adding ID columns:", e)

# ✅ Step 2.3: Create Flags for Filtering Vehicles
try:
    vehicles_df["Is_Special"] = vehicles_df["Vehicle_Type"].apply(lambda x: 1 if "Special" in x else 0)
    print("✅ Flagged special vehicles.")
except Exception as e:
    print("❌ Error flagging vehicle types:", e)

# ✅ Step 2.4: Feature Checkpoint
try:
    print("\n📦 Orders Sample:")
    print(orders_df[["Order_ID", "Product_ID", "Fragile_Flag", "Temp_Sensitive_Flag", "Total_Weight", "Total_Volume"]].head(3))

    print("\n🚛 Vehicles Sample:")
    print(vehicles_df[["Vehicle_ID", "Vehicle_Type", "Weight_Capacity (kg)", "Volume_Capacity (L)", "Is_Special"]].head(3))
except Exception as e:
    print("❌ Preview failed:", e)


✅ DataFrames copied.
✅ ID columns added.
✅ Flagged special vehicles.

📦 Orders Sample:
   Order_ID        Product_ID  Fragile_Flag  Temp_Sensitive_Flag  \
0         0  DEW-1000-B31-795             0                    0   
1         1  IMM-1001-B59-384             0                    1   
2         2  SUP-1002-B44-434             0                    0   

   Total_Weight  Total_Volume  
0          22.2          9.25  
1          37.0         22.20  
2          50.0         24.00  

🚛 Vehicles Sample:
  Vehicle_ID Vehicle_Type  Weight_Capacity (kg)  Volume_Capacity (L)  \
0  GJ08X2709      General                  9000                35000   
1  GJ30X2371  Specialised                  4000                15000   
2  GJ24X8239      General                  9000                35000   

   Is_Special  
0           0  
1           1  
2           0  


In [9]:
import numpy as np

class VehicleAssignmentEnv:
    def __init__(self, orders_df, vehicles_df):
        self.orders_df = orders_df.reset_index(drop=True)
        self.vehicles_df = vehicles_df.copy()

        self.num_orders = len(self.orders_df)
        self.num_vehicles = len(self.vehicles_df)

        self.current_order_idx = 0
        self.reset()

    def reset(self):
        self.current_order_idx = 0
        self.vehicles_df["Remaining_Weight"] = self.vehicles_df["Weight_Capacity (kg)"]
        self.vehicles_df["Remaining_Volume"] = self.vehicles_df["Volume_Capacity (L)"]
        self.vehicles_df["Used"] = False

        return self.get_state()

    def get_state(self):
        order = self.orders_df.iloc[self.current_order_idx]
        state = {
            "order_weight": order["Total_Weight"],
            "order_volume": order["Total_Volume"],
            "fragile": order["Fragile_Flag"],
            "temp_sensitive": order["Temp_Sensitive_Flag"],
            "vehicles": self.vehicles_df[["Remaining_Weight", "Remaining_Volume", "Is_Special"]].values
        }
        return state

    def step(self, action_idx):
        """
        action_idx: index of selected vehicle from vehicles_df
        """

        reward = 0
        done = False

        order = self.orders_df.iloc[self.current_order_idx]
        vehicle = self.vehicles_df.iloc[action_idx]

        weight_needed = order["Total_Weight"]
        volume_needed = order["Total_Volume"]

        # Check if vehicle can handle
        if vehicle["Remaining_Weight"] >= weight_needed and vehicle["Remaining_Volume"] >= volume_needed:
            self.vehicles_df.at[action_idx, "Remaining_Weight"] -= weight_needed
            self.vehicles_df.at[action_idx, "Remaining_Volume"] -= volume_needed
            self.vehicles_df.at[action_idx, "Used"] = True

            # Reward = utilization score + vehicle suitability
            weight_util_score = weight_needed / vehicle["Weight_Capacity (kg)"]
            volume_util_score = volume_needed / vehicle["Volume_Capacity (L)"]

            vehicle_type_penalty = 0
            if (order["Fragile_Flag"] or order["Temp_Sensitive_Flag"]) and not vehicle["Is_Special"]:
                vehicle_type_penalty = -0.5  # Penalize bad vehicle

            reward = weight_util_score + volume_util_score + vehicle_type_penalty
        else:
            reward = -1  # Penalize if not fitting

        # Move to next order
        self.current_order_idx += 1
        if self.current_order_idx >= self.num_orders:
            done = True

        return self.get_state(), reward, done


In [10]:
env = VehicleAssignmentEnv(orders_df, vehicles_df)
state = env.reset()
print("🧾 Initial State Sample:")
print("Order Weight:", state["order_weight"])
print("Vehicle Pool (first 3):", state["vehicles"][:3])

next_state, reward, done = env.step(1)  # Try assigning second vehicle
print("🎯 Reward:", reward)
print("✅ Done:", done)
print("🔁 Next Order Weight:", next_state["order_weight"])



🧾 Initial State Sample:
Order Weight: 22.2
Vehicle Pool (first 3): [[ 9000 35000     0]
 [ 4000 15000     1]
 [ 9000 35000     0]]
🎯 Reward: 0.006166666666666667
✅ Done: False
🔁 Next Order Weight: 37.0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19668\898852749.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3977.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.vehicles_df.at[action_idx, "Remaining_Weight"] -= weight_needed
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19668\898852749.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14990.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.vehicles_df.at[action_idx, "Remaining_Volume"] -= volume_needed


In [15]:
import random

num_episodes = 5

for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False  # ✅ Initialize done
    print(f"\n🎬 Episode {episode+1} Start ----------------------")

    while not done:
        action = random.randint(0, env.num_vehicles - 1)

        try:
            next_state, reward, done = env.step(action)  # ✅ Update done here
            total_reward += reward

            print(f"🚀 Order {env.current_order_index}: Vehicle {action} ➜ Reward: {reward:.4f}, Done: {done}")
        
        except Exception as e:
            print(f"❌ ERROR during step: {str(e)}")
            break

    print(f"✅ Episode {episode+1} Finished with Total Reward: {total_reward:.4f}")



🎬 Episode 1 Start ----------------------
❌ ERROR during step: 'VehicleAssignmentEnv' object has no attribute 'current_order_index'
✅ Episode 1 Finished with Total Reward: 0.0062

🎬 Episode 2 Start ----------------------
❌ ERROR during step: 'VehicleAssignmentEnv' object has no attribute 'current_order_index'
✅ Episode 2 Finished with Total Reward: 0.0062

🎬 Episode 3 Start ----------------------
❌ ERROR during step: 'VehicleAssignmentEnv' object has no attribute 'current_order_index'
✅ Episode 3 Finished with Total Reward: 0.0027

🎬 Episode 4 Start ----------------------
❌ ERROR during step: 'VehicleAssignmentEnv' object has no attribute 'current_order_index'
✅ Episode 4 Finished with Total Reward: 0.0062

🎬 Episode 5 Start ----------------------
❌ ERROR during step: 'VehicleAssignmentEnv' object has no attribute 'current_order_index'
✅ Episode 5 Finished with Total Reward: 0.0027


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19668\898852749.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3977.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.vehicles_df.at[action_idx, "Remaining_Weight"] -= weight_needed
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19668\898852749.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14990.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.vehicles_df.at[action_idx, "Remaining_Volume"] -= volume_needed
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19668\898852749.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3977.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype